# Standard logic: 'U', 'Z', 'X' and others

The standard logic packages from VHDL allow to simulate other signal states than the default True or False. In some cases, simulation entities may want to make use of them, however, the MyHDL compatible `intbv` type does not support it.

Therefore, the `StandardLogicSignal` functions as extension to the default signal types and can be used to interface legacy VHDL modules where type conversion is required. A limited support for Verilog is present, however not thoroughly tested. Some constructs may not be portable.

The specific `TristateSignal` class is another higher level construct to switch `inout` ports according to a somewhat strict design rule.

## Standard logic vectors

In [1]:
from myirl.library.stdlogic import StandardLogicSignal

s = StandardLogicSignal("11UU11")
t = StandardLogicSignal("10UUUU")
a = StandardLogicSignal(8 * 'Z')

The internal properties of the `stdlogic.slbv` data type provide a bit of evaluation for multi state bit logic, so constructs like the following output as:

In [2]:
expr = s & t
expr.evaluate()

SLV(10XXXX)

Internally, conversion between `intbv` and `slbv` is casted, however not within interface mapping. Explicit `slbv` signals must be registered for external object wrappers.

## Tri state signals

Tristate signals are meant to represent a tri state bus driver at the peripheral edge. They must never occur inside a hardware design, for bidirectional communication, use Port signal classes.

A TriStateSignal is initialized with 'weak' values that are actually overriden by its `.driver()`. It can simultaneously be read from and driven by the internal driver.

### Initialization

You can only initialize with:
* `L`: Pulldown
* `H`: Pullup
* `Z`: High Z

In [3]:
from myirl.library.tristate import *
from cyhdl import *

ts = TristateSignal("LHZZZZZZZ", name = "tristate_inout")
ts

<tristate_inout>

The internal driver id is however differen, because it is in fact a different signal:

In [4]:
ts.driver()

<tristate_inout_drv>

The actual decision whether the `TristateSignal` is  driven, is made by a `.select` statement, which (in a HDL language) typically infers into a simple ternary if assignment which is understood by most Simulators.

The `sel` line below functions as drive enable.

In [5]:
data = Signal(intbv()[9:], name = "data")
sel = Signal(bool(), "en")

m = ts.select(data, sel)

We elaborate to VHDL to confirm its function:

In [6]:
from myirl.targets.dummy import DummyTargetModule

d = DummyTargetModule()
m.emit(d)

tristate_inout <= STD_LOGIC_VECTOR(data) when (en = '1') else "LHZZZZZZZ";


## Example

To make use of the TristateSignal in a top level unit, you typically specify a `BidirectionalSignal` data type.

In this example, we also model a delay to mimic bus behaviour.

**Important**: The main sequential stimulation routine should be called `main()` in order to output a VCD trace for Verilog. For VHDL, this is not relevant, but a `wavetrace` argument will be taken instead to the Simulator call.

In [7]:
@block
def unit(ts : BidirectionalSignal, sel : Signal, data : Signal):
    idata = data.clone('idata')
    
    @always(delay(2))
    def wireup():
        ts.select(data, sel) # A macro behind the curtains

    connections = [
        idata.wireup(ts)
    ]

    return instances()

@block
def tb():
    t = TristateSignal(8 * 'Z')
    sample_clk = ClkSignal(name="sample_clk")
    sel = Signal(bool())
    data = Signal(intbv()[8:])
    
    u = unit(t, sel, data)
    
    @always(delay(1))
    def clkgen():
        sample_clk.set(~sample_clk)
    
    @sequence
    def main():
        sel.next = False
        data.next = 0xaa
        yield delay(5)
        sel.next = True
        yield 3 * (sample_clk.posedge,)
        assert t == 0xaa
        sel.next = False
        yield 2 * (sample_clk.posedge,)
        assert t != 0xaa, "Failed Z"
        yield 2 * (sample_clk.posedge,)
        data.next = 0xbb
        sel.next = True
        yield 2 * (sample_clk.posedge,)
        assert t == 0xbb, "Failed 0xbb"
        yield delay(4)
        sel.next = False
        yield delay(8),

        raise StopSimulation
    
    
    return locals()

For more internals, check [TristateSignals](../../notebooks/tristate.ipynb)

## Simulation of example

This will simulate for both VHDL and Verilog targets, however only the VHDL output precisely shows the tristate behaviour.

In [10]:
from myirl.test.common_test import Simulator

def test():
    t = tb()
    s = Simulator(targets.VHDL)
    s.run(t, 200, debug = False, wavetrace = 'tri.vcd')
    
    return s.used_files
f = test()

 Module tb: Existing instance tb, rename to tb_1 
 Module tb: Existing instance unit, rename to unit_1 
 Writing 'unit_1' to file /tmp/myirl_tb_p159x5j9/unit_1.vhdl 
 Writing 'tb_1' to file /tmp/myirl_tb_p159x5j9/tb_1.vhdl 
 Creating library file module_defs.vhdl 


A rudimentary waveform display to show the simulation output (clock synchronous):

In [9]:
import wavedraw; import nbwavedrom
TB = tb.name;

waveform = wavedraw.vcd2wave("tri.vcd", TB + '.sample_clk', None)
nbwavedrom.draw(waveform)